# Q1. MITM

## Our solution
Man-in-the-Middle (MITM) attack on the Diffie-Hellman key exchange between Alice and Bob.

Our attack script, mitm.py, acts as a transparent proxy with malicious intent:
1.  Interception: It listens on two specific ports to intercept connections intended for Alice and Bob.
2.  Key Replacement:
    - When Bob connects, mitm.py intercepts his public key ($g^y$) and sends him a fake public key ($g^z$).
    - When Alice connects, mitm.py sends her the fake public key ($g^z$) and intercepts her real public key ($g^x$).
3.  Result: Alice and Bob believe they have a secure connection with each other, but they have actually established separate shared secrets with the attacker.

## The Execution Instructions
The code can be found in the Q1 folder.

Please execute the files in separate terminals in the following order.

### Run the Scripts
Open your terminal and run the scripts in this specific order to ensure listeners are active before connections are attempted:

1.  Start Alice: (Alice waits for a connection, so she must start first)
    ```bash
    python3 alice.py
    ```
2.  Start the Attacker:
    ```bash
    python3 mitm.py
    ```
3.  Start Bob: (Bob initiates the connection cycle)
    ```bash
    python3 bob.py
    ```

#### 3. Expected Output
In the mitm.py terminal window, you will see:
* Interception of GY from Bob.
* Interception of GX from Alice.
* Success Message: Displaying the two compromised shared secrets (one shared with Alice, one shared with Bob).

# Q2. ECC

### P1

**Proof:**
The verification algorithm checks if m = σ + k · pkA. We can prove this is always true for a valid signature by substituting the definitions of `σ` and pkA from the signing process into the verification equation.

1.  Start with the right-hand side of the verification formula:
    σ + k · pkA

2.  Substitute σ = m - k · skA · G and pkA = skA · G:
    = (m - k · skA · G) + k · (skA · G)

3.  The terms (- k · skA · G) and (+ k · skA · G) are additive inverses and cancel out, leaving:
    = m

Since the expression simplifies to m, the verification check m = m is always true. Thus, the scheme is correct for validly signed messages.

### P2

**Forgery Technique:**
The scheme is insecure because an attacker can forge a signature for any arbitrary message m' without the private key skA.

An attacker performs the following steps:
1.  **Choose any message m'** they want to forge.
2.  **Choose any random scalar k'**. For instance, let k' = 1.
3.  **Calculate the forged signature σ'** by rearranging the public verification formula m' = σ' + k' · pkA to solve for σ':
    σ' = m' - k' · pkA

This calculation only requires the desired message m', the chosen k', and the public key pkA, all of which are known to the attacker. The private key skA is not needed.

The resulting tuple (m', k', σ') will always pass verification, as (m' - k' · pkA) + k' · pkA simplifies to m', thus satisfying the check.

# Q3. ElGamal

### Question - P1:
In the decryption function, we have the creation of the shared key $s$ and we have that the ciphertext $c = m \cdot s$.  
By performing:
$$
m = c ⋅ s^{-1} = m ⋅ s ⋅ s^{-1} = m ⋅ 1 = m
$$
Thus, we obtain the original message.

### Question - P2:
To be able to obtain the message: $m \leftarrow c \cdot s^{-1}$, we must know the value of $s$, where:  
$$
s = g^{xy}
$$  
To be able to break the confidentiality of ElGamal, an attacker would have to discover the shared key $s$ from $X = g^{x}$ (public key) and $Y = g^{y}$ (random value sent in the ciphertext).
Computing $s = g^{xy}$ using only $g^{x}$ and $g^{y}$ corresponds exactly to the Diffie–Hellman problem. If the attacker were able to solve the discrete logarithm, that is, discover the exponent $x$ such that $X = g^{x}$, then they could simply compute $s = Y^{x}$ and obtain the original message. Therefore, the message would be compromised.

The practical security of ElGamal, in the sense of guaranteeing confidentiality, depends on the difficulty of the CDH (Computational Diffie–Hellman) and DDH (Decisional Diffie–Hellman) problems.
If CDH is difficult, then it is not possible to discover any information about $g^{xy}$.
Furthermore, if the DDH problem is also difficult, this means that an attacker cannot distinguish between $g^{x}$, $g^{y}$, and $g^{xy}$.
Thus, $c = m \cdot g^{xy}$ does not reveal any useful information about the message.

### Question - P3:

Let:
- $x$ is secret key
- $pub\_k = g^{x}$; public key
- ciphertext for message m:
$$
c = (c1, c2) = (g^{y}, m \cdot pub\_k^{y})
$$
- y is randomly chosen for encryption


Now we want a new ciphertext $c'$ such that the decrypted message $m'$ is related to $m$.

Now, we pick a factor $k \in \mathbb{Z}_q^*$  

Modify the ciphertext: 
$$
c' = (c1', c2') = (c1, k \cdot c2​) = (g^{y},k \cdot m \cdot pub\_k^{y})
$$

To decrypt $c'$:

$$
m' = \frac{c_2'}{(c_1')^x} = \frac{k \cdot m \cdot pub\_k^y}{(g^y)^x} = \frac{k \cdot m \cdot g^{xy}}{g^{xy}} = k \cdot m
$$

Conclusion: The new plaintext is exactly $m' = k ⋅ m$, which is a predictable function of the original message.
